# 점진적 학습 알고리즘 : 확률적 경사 하강법
![](./확률적경사하강법.PNG)

## 확률적 경사 하강법
- 손실 함수 : 어떤 문제에서 머닝러신 알고리즘이 얼마나 엉터리인지 측정하는 기준
- 경사 하강법에서 내려오는 산이 손실 함수
- 이진 분류에서 사용하는 손실 함수 : 로지스틱 손실 함수 또는 이진 크로스엔트로피 손실 함수
- 다중 분류에서 사용하는 손실 함수 : 크로스엔트로피 손실 함수
- -log(예측확률) 또는 -log(1-예측확률)을 사용
- 손실 함수는 우리가 만들지 않아도 됨
- scikit-learn의 LogisticRegression이 로지스틱 회귀를 위한 클래스
- predict_proba() 메서드는 예측 확률을 반환
  - 이진 분류 : 샘플마다 음성 클래스와 양성 클래스에 대한 확률 반환
  - 다중 분류 : 샘플마다 모든 클래스에 대한 확률 반환
- decision_function()은 모델이 학습한 선형 방정식의 출력을 반환
  - 이진 분류 : 양성 클래스의 선형 방정식
  - 다중 분류 : 각 클래스마다 선형 방정식